In [1]:
import numpy as np
from tqdm import tqdm
import pymc as pm
import skfem as fem
from skfem import MeshLine, ElementLineP1, Basis, BilinearForm, LinearForm
from skfem.helpers import dot, grad, d, dd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import truncnorm
import scipy.stats as scstats
import scipy.sparse as scsparse
import scipy.optimize as scoptimize
import scipy.linalg as sclinalg

# import pytensor.tensor.subtensor as subtensor
# import pytensor.tensor as tensor
# import pytensor.sparse as sparsetensor
import arviz as az

import pytensor.tensor as at
from pytensor import function, scan, shared, config, printing
from pytensor import grad as ptgrad
import numpy as np
from tqdm import tqdm
from pytensor import sparse

import scienceplots
# plot params
contour_levels = 10
plt.style.use(['science', 'grid'])
color_list = [(253, 231, 37),(194, 223, 35),(134, 213, 73),(82, 197, 105),(42, 176, 127),(30, 155, 138),(37, 133, 142),(45, 112, 142),(56, 88, 140),(67, 62, 133),(72, 33, 115),(68, 1, 84)]
color_list = [tuple(ti/255 for ti in t) for t in color_list]

In [9]:
import sympy as sp

# Define spatial and temporal variables
x, t = sp.symbols('x t')

g = sp.symbols(r'g')
s = sp.symbols(r's')
h_bar = sp.symbols(r'\bat{H}')
nu = sp.symbols(r'\nu')
theta = sp.symbols(r'\theta')
dt = sp.symbols(r'\Delta_t')

u = sp.Function(r'u')(x,t)
eta = sp.Function(r'\eta')(x,t)
b = sp.Function(r'b')(x) #5*(1+sp.tanh((x-5)/2000))
H = h_bar - b

EQ1 = u*sp.diff(u, x) - nu*sp.diff(u, x, 2) + g*sp.diff(eta, x)
EQ2 = sp.diff(H, x)*sp.diff(u, x, 2) + sp.diff(H, x, 2)*sp.diff(u, x) + sp.diff(eta, x, 2)*sp.diff(u, x) + sp.diff(eta, x)*sp.diff(u, x, 2)

u_1 = sp.Function(r'u^{n+1}')(x)
u_0 = sp.Function(r'u^{n}')(x)

eta_1 = sp.Function(r'\eta^{n+1}')(x)
eta_0 = sp.Function(r'\eta^{n}')(x)

cn_u = (u_1 - u_0) / dt
cn_eta = (eta_1 - eta_0) / dt

cn_EQ1 = -cn_u + theta*EQ1.subs([(u, u_1), (eta, eta_1)]) + (1-theta)*EQ1.subs([(u, u_0), (eta, eta_0)]).expand().doit()
cn_EQ2 = -cn_eta + theta*EQ2.subs([(u, u_1), (eta, eta_1)]) + (1-theta)*EQ2.subs([(u, u_0), (eta, eta_0)]).expand().doit()


In [10]:
EQ1

-\nu*Derivative(u(x, t), (x, 2)) + g*Derivative(\eta(x, t), x) + u(x, t)*Derivative(u(x, t), x)

In [11]:
cn_EQ2

\theta*(Derivative(\eta^{n+1}(x), x)*Derivative(u^{n+1}(x), (x, 2)) + Derivative(\eta^{n+1}(x), (x, 2))*Derivative(u^{n+1}(x), x) - Derivative(b(x), x)*Derivative(u^{n+1}(x), (x, 2)) - Derivative(b(x), (x, 2))*Derivative(u^{n+1}(x), x)) + (1 - \theta)*(Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), (x, 2)) + Derivative(\eta^{n}(x), (x, 2))*Derivative(u^{n}(x), x) - Derivative(b(x), x)*Derivative(u^{n}(x), (x, 2)) - Derivative(b(x), (x, 2))*Derivative(u^{n}(x), x)) - (\eta^{n+1}(x) - \eta^{n}(x))/\Delta_t

In [167]:
phi = sp.Function(r'\varphi')(x)
psi = sp.Function(r'\psi')(x)

def multiply_terms(expr, k_func, j_func):
    new_expr = 0
    for term in expr.as_ordered_terms():
        # Check for velocity terms
        if any([term.has(u_1), term.has(u_0)]):
            new_expr += k_func * term
        # Check for pressure terms
        elif any([term.has(eta_1), term.has(eta_0)]):
            new_expr += j_func * term
        else:
            new_expr += term
    return new_expr

cn_EQ1_modified = multiply_terms(cn_EQ1.expand().doit(), phi, psi).expand().doit()
cn_EQ2_modified = multiply_terms(cn_EQ2.expand().doit(), phi, psi).expand().doit()

In [168]:
cn_EQ1.expand().doit()

-\nu*\theta*Derivative(u^{n+1}(x), (x, 2)) + \nu*\theta*Derivative(u^{n}(x), (x, 2)) - \nu*Derivative(u^{n}(x), (x, 2)) + \theta*g*Derivative(\eta^{n+1}(x), x) - \theta*g*Derivative(\eta^{n}(x), x) + \theta*u^{n+1}(x)*Derivative(u^{n+1}(x), x) - \theta*u^{n}(x)*Derivative(u^{n}(x), x) + g*Derivative(\eta^{n}(x), x) + u^{n}(x)*Derivative(u^{n}(x), x) - u^{n+1}(x)/\Delta_t + u^{n}(x)/\Delta_t

In [175]:
cn_EQ2_modified

\theta*\varphi(x)*Derivative(\eta^{n+1}(x), x)*Derivative(u^{n+1}(x), (x, 2)) + \theta*\varphi(x)*Derivative(\eta^{n+1}(x), (x, 2))*Derivative(u^{n+1}(x), x) - \theta*\varphi(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), (x, 2)) - \theta*\varphi(x)*Derivative(\eta^{n}(x), (x, 2))*Derivative(u^{n}(x), x) - \theta*\varphi(x)*Derivative(b(x), x)*Derivative(u^{n+1}(x), (x, 2)) + \theta*\varphi(x)*Derivative(b(x), x)*Derivative(u^{n}(x), (x, 2)) - \theta*\varphi(x)*Derivative(b(x), (x, 2))*Derivative(u^{n+1}(x), x) + \theta*\varphi(x)*Derivative(b(x), (x, 2))*Derivative(u^{n}(x), x) + \varphi(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), (x, 2)) + \varphi(x)*Derivative(\eta^{n}(x), (x, 2))*Derivative(u^{n}(x), x) - \varphi(x)*Derivative(b(x), x)*Derivative(u^{n}(x), (x, 2)) - \varphi(x)*Derivative(b(x), (x, 2))*Derivative(u^{n}(x), x) - \eta^{n+1}(x)*\psi(x)/\Delta_t + \eta^{n}(x)*\psi(x)/\Delta_t

In [181]:
sp.diff(eta_1,x,2)*psi

\psi(x)*Derivative(\eta^{n+1}(x), (x, 2))

In [170]:
n = sp.Function(r'n')(x)

greened_EQ1 = cn_EQ1_modified.subs([(sp.diff(u_0,x,2)*phi, sp.diff(u_0,x,1)*phi*n -sp.diff(u_0,x,1)*sp.diff(phi,x,1)),
                      (sp.diff(u_1,x,2)*phi, sp.diff(u_1,x,1)*phi*n -sp.diff(u_1,x,1)*sp.diff(phi,x,1)),
                      (sp.diff(eta_0,x,2)*psi, sp.diff(eta_0,x,1)*psi*n -sp.diff(eta_0,x,1)*sp.diff(psi,x,1)),
                      (sp.diff(eta_1,x,2)*psi, sp.diff(eta_1,x,1)*psi*n -sp.diff(eta_1,x,1)*sp.diff(psi,x,1))
                      ]).expand().doit()

greened_EQ2 = cn_EQ2_modified.subs([(sp.diff(u_0,x,2)*phi, sp.diff(u_0,x,1)*phi*n -sp.diff(u_0,x,1)*sp.diff(phi,x,1)),
                      (sp.diff(u_1,x,2)*phi, sp.diff(u_1,x,1)*phi*n -sp.diff(u_1,x,1)*sp.diff(phi,x,1)),
                      (sp.diff(eta_0,x,2)*psi, sp.diff(eta_0,x,1)*psi*n -sp.diff(eta_0,x,1)*sp.diff(psi,x,1)),
                      (sp.diff(eta_1,x,2)*psi, sp.diff(eta_1,x,1)*psi*n -sp.diff(eta_1,x,1)*sp.diff(psi,x,1))
                      ]).expand().doit()

In [171]:
cn_EQ1.expand().doit()

-\nu*\theta*Derivative(u^{n+1}(x), (x, 2)) + \nu*\theta*Derivative(u^{n}(x), (x, 2)) - \nu*Derivative(u^{n}(x), (x, 2)) + \theta*g*Derivative(\eta^{n+1}(x), x) - \theta*g*Derivative(\eta^{n}(x), x) + \theta*u^{n+1}(x)*Derivative(u^{n+1}(x), x) - \theta*u^{n}(x)*Derivative(u^{n}(x), x) + g*Derivative(\eta^{n}(x), x) + u^{n}(x)*Derivative(u^{n}(x), x) - u^{n+1}(x)/\Delta_t + u^{n}(x)/\Delta_t

In [172]:
greened_EQ1

-\nu*\theta*\varphi(x)*n(x)*Derivative(u^{n+1}(x), x) + \nu*\theta*\varphi(x)*n(x)*Derivative(u^{n}(x), x) + \nu*\theta*Derivative(\varphi(x), x)*Derivative(u^{n+1}(x), x) - \nu*\theta*Derivative(\varphi(x), x)*Derivative(u^{n}(x), x) - \nu*\varphi(x)*n(x)*Derivative(u^{n}(x), x) + \nu*Derivative(\varphi(x), x)*Derivative(u^{n}(x), x) + \theta*g*\psi(x)*Derivative(\eta^{n+1}(x), x) - \theta*g*\psi(x)*Derivative(\eta^{n}(x), x) + \theta*\varphi(x)*u^{n+1}(x)*Derivative(u^{n+1}(x), x) - \theta*\varphi(x)*u^{n}(x)*Derivative(u^{n}(x), x) + g*\psi(x)*Derivative(\eta^{n}(x), x) + \varphi(x)*u^{n}(x)*Derivative(u^{n}(x), x) - \varphi(x)*u^{n+1}(x)/\Delta_t + \varphi(x)*u^{n}(x)/\Delta_t

In [173]:
greened_EQ2

\theta*\varphi(x)*n(x)*Derivative(\eta^{n+1}(x), x)*Derivative(u^{n+1}(x), x) - \theta*\varphi(x)*n(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) - \theta*\varphi(x)*n(x)*Derivative(b(x), x)*Derivative(u^{n+1}(x), x) + \theta*\varphi(x)*n(x)*Derivative(b(x), x)*Derivative(u^{n}(x), x) + \theta*\varphi(x)*Derivative(\eta^{n+1}(x), (x, 2))*Derivative(u^{n+1}(x), x) - \theta*\varphi(x)*Derivative(\eta^{n}(x), (x, 2))*Derivative(u^{n}(x), x) - \theta*\varphi(x)*Derivative(b(x), (x, 2))*Derivative(u^{n+1}(x), x) + \theta*\varphi(x)*Derivative(b(x), (x, 2))*Derivative(u^{n}(x), x) - \theta*Derivative(\eta^{n+1}(x), x)*Derivative(\varphi(x), x)*Derivative(u^{n+1}(x), x) + \theta*Derivative(\eta^{n}(x), x)*Derivative(\varphi(x), x)*Derivative(u^{n}(x), x) + \theta*Derivative(\varphi(x), x)*Derivative(b(x), x)*Derivative(u^{n+1}(x), x) - \theta*Derivative(\varphi(x), x)*Derivative(b(x), x)*Derivative(u^{n}(x), x) + \varphi(x)*n(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) - \va

In [36]:
u_hat = sp.Function(r'\hat{u}')(x)
eta_hat = sp.Function(r'\hat{\eta}')(x)

epsilon = sp.symbols(r'\epsilon')

u_perturbed = u_1 + epsilon*u_hat
eta_perturbed = eta_1 + epsilon*eta_hat

u_perturbed_x = sp.diff(u_perturbed, x)

v_perturbed_x = sp.diff(eta_perturbed, x)


J11 = greened_EQ1.subs([(u_1, u_perturbed), (sp.diff(u,x,1), u_perturbed_x)])
J12 = greened_EQ1.subs([(eta_1, eta_perturbed), (sp.diff(eta,x,1), v_perturbed_x)])

J21 = greened_EQ2.subs([(u_1, u_perturbed), (sp.diff(u,x,1), u_perturbed_x)])
J22 = greened_EQ2.subs([(eta_1, eta_perturbed), (sp.diff(eta,x,1), v_perturbed_x)])

J11_lim = sp.limit((J11 - greened_EQ1) / epsilon, epsilon, 0)
J12_lim = sp.limit((J12 - greened_EQ1) / epsilon, epsilon, 0)

J21_lim = sp.limit((J21 - greened_EQ2) / epsilon, epsilon, 0)
J22_lim = sp.limit((J22 - greened_EQ2) / epsilon, epsilon, 0)


In [37]:
J11_lim

(-\Delta_t*\nu*\theta*\varphi(x)*n(x)*Derivative(\hat{u}(x), x) + \Delta_t*\nu*\theta*Derivative(\hat{u}(x), x)*Derivative(\varphi(x), x) + \Delta_t*\theta*\hat{u}(x)*\varphi(x)*Derivative(u^{n+1}(x), x) + \Delta_t*\theta*\varphi(x)*u^{n+1}(x)*Derivative(\hat{u}(x), x) - \hat{u}(x)*\varphi(x))/\Delta_t

In [87]:
equation = greened_EQ1

S = sp.symbols('S')   # Boundary differential

integrand_domain1 = sum(
    sp.Integral(term, x)  # Domain integral with differential dx
    for term in equation.args
    if not term.has(n)  # Exclude terms involving n
)

# Boundary integrals (terms involving n)
integrand_boundary1 = sum(
    sp.Integral(term, S)  # Boundary integral with differential dS
    for term in equation.args
    if term.has(n)
)

# The final weak form with symbolic integrals
final_expression1 = integrand_domain1 + integrand_boundary1

equation = greened_EQ2

S = sp.symbols('S')   # Boundary differential

integrand_domain2 = sum(
    sp.Integral(term, x)  # Domain integral with differential dx
    for term in equation.args
    if not term.has(n)  # Exclude terms involving n
)

# Boundary integrals (terms involving n)
integrand_boundary2 = sum(
    sp.Integral(term, S)  # Boundary integral with differential dS
    for term in equation.args
    if term.has(n)
)

# The final weak form with symbolic integrals
final_expression2 = integrand_domain2 + integrand_boundary2

In [96]:
sp.collect(integrand_domain1, [phi, psi]).doit()

\nu*\theta*Integral(Derivative(\varphi(x), x)*Derivative(u^{n+1}(x), x), x) - \nu*\theta*Integral(Derivative(\varphi(x), x)*Derivative(u^{n}(x), x), x) + \nu*Integral(Derivative(\varphi(x), x)*Derivative(u^{n}(x), x), x) + \theta*g*Integral(\psi(x)*Derivative(\eta^{n+1}(x), x), x) - \theta*g*Integral(\psi(x)*Derivative(\eta^{n}(x), x), x) + \theta*Integral(\varphi(x)*u^{n+1}(x)*Derivative(u^{n+1}(x), x), x) - \theta*Integral(\varphi(x)*u^{n}(x)*Derivative(u^{n}(x), x), x) + g*Integral(\psi(x)*Derivative(\eta^{n}(x), x), x) + Integral(\varphi(x)*u^{n}(x)*Derivative(u^{n}(x), x), x) - Integral(\varphi(x)*u^{n+1}(x), x)/\Delta_t + Integral(\varphi(x)*u^{n}(x), x)/\Delta_t

In [86]:
mat_exp_1 = 0
term_count = 0
n_u = sp.symbols(r'n_u')
n_eta = sp.symbols(r'n_eta')
M = sp.MatrixSymbol(r'\mathbf{M}', n_u, n_u)
A = sp.MatrixSymbol(r'\mathbf{A}', n_eta, n_eta)
u_1_vec = sp.Symbol(r'\mathbf{u}^{n+1}')
u_0_vec = sp.Symbol(r'\mathbf{u}^{n}')
eta_1_vec = sp.Symbol(r'\mathbf{\eta}^{n+1}')
eta_0_vec = sp.Symbol(r'\mathbf{\eta}^{n}')

for term in integrand_domain.args:
    term_check = []
    if term.has(phi):
        # print(term)
        term_check.append(0)
        subterm_count = 0
        term_count += 1
        u_1_check = term_check.append(int(term.has(u_1)))
        u_0_check = term_check.append(int(term.has(u_0)))
        diff_u_1_check = term_check.append(int(term.has(sp.diff(u_1,x))))
        diff_u_0_check = term_check.append(int(term.has(sp.diff(u_0,x))))
        print(term_check)
        # for subterm in term.atoms(sp.Derivative):
        #     # print(subterm)
        #     subterm_count += 1
        # print(f'Term {term_count} has a phi and a u_1: {u_1_check}, or u_0: {u_0_check}, and has {subterm_count} derivative terms.')
    elif term.has(psi):
        # print(term)
        term_check.append(1)
        subterm_count = 0
        term_count += 1
        eta_1_check = term_check.append(int(term.has(eta_1)))
        eta_0_check = term_check.append(int(term.has(eta_0)))
        diff_eta_1_check = term_check.append(int(term.has(sp.diff(eta_1,x))))
        diff_eta_0_check = term_check.append(int(term.has(sp.diff(eta_0,x))))
        # for subterm in term.atoms(sp.Derivative):
        #     # print(subterm)
        #     subterm_count += 1
        # print(f'Term {term_count} has a psi and a eta_1: {u_1_check}, or eta_0: {u_0_check}, and has {subterm_count} derivative terms.')
    print(term_check)
    
    

[0, 0, 1, 0, 1]
[0, 0, 1, 0, 1]
[1, 0, 1, 0, 1]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 1]
[0, 0, 1, 0, 1]
[0, 1, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 1, 0, 1, 0]
[0, 1, 0, 1, 0]
[1, 1, 0, 1, 0]
[0, 1, 0, 1, 0]
[0, 1, 0, 1, 0]
[0, 0, 1, 0, 1]
[0, 0, 1, 0, 1]
[1, 0, 1, 0, 1]
[0, 0, 1, 0, 1]
[0, 0, 1, 0, 1]


In [126]:
horizontal_cell_count = 500
space_start = 0
space_end = 10000
xs = np.linspace(space_start, space_end, horizontal_cell_count)

u_mesh = fem.MeshLine(xs)
element1 = fem.ElementLineP2()
element2 = fem.ElementLineP1()

basis_u = fem.Basis(u_mesh, element1)
basis_eta = fem.Basis(u_mesh, element2)

u_coords = basis_u.doflocs[0].T
eta_coords = basis_eta.doflocs[0].T

u_b1 = np.where(u_coords == space_start)[0][0]
u_b2 = np.where(u_coords == space_end)[0][0]
eta_b1 = np.where(eta_coords == space_start)[0][0]
eta_b2 = np.where(eta_coords == space_end)[0][0]

In [159]:
nu = .1
g = -9.8
h_bar = 30
s = 2000

def b_func(x, s):
    return 5*(1+np.tanh((x - s)/2000))

def tau_func(t):
    return 2*(1+np.cos((4*np.pi*t)/(86400)))

t = 0
theta = 0.6
dt = 1 
t_end = 12*60*60. 
time_range = np.linspace(0, t_end, int(t_end // dt)+1)
time_steps = len(time_range)

u_fields = np.zeros((time_steps, len(u_coords)))
eta_fields = np.zeros((time_steps, len(eta_coords)))

eta_fields[:, 0] = tau_func(time_range)

In [160]:
@fem.LinearForm
def F_1_1(v,w):
    C1 = v*(theta*w.u_1*grad(w.u_1)[0] - theta*w.u_0*grad(w.u_0)[0] + w.u_0*grad(w.u_0)[0] - (1/dt)*(w.u_1 - w.u_0))
    C2 = grad(v)[0]*(nu*theta*w.u_1[0] - nu*theta*grad(w.u_0)[0] + nu*grad(w.u_0)[0])
    return C1 + C2

@fem.LinearForm
def F_1_2(v,w):
    return v*(theta*g*grad(w.eta_1)[0] - theta*g*grad(w.eta_0)[0] + g*grad(w.eta_0)[0])

@fem.LinearForm
def F_2_1(v,w):
    C1 = v*()


array([[4.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.99999998, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.99999992, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [3.99999992, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.99999998, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [4.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [174]:
sp.collect(greened_EQ2, [phi, psi])

(-\eta^{n+1}(x)/\Delta_t + \eta^{n}(x)/\Delta_t)*\psi(x) + (-\theta*Derivative(\eta^{n+1}(x), x)*Derivative(u^{n+1}(x), x) + \theta*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) + \theta*Derivative(b(x), x)*Derivative(u^{n+1}(x), x) - \theta*Derivative(b(x), x)*Derivative(u^{n}(x), x) - Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) + Derivative(b(x), x)*Derivative(u^{n}(x), x))*Derivative(\varphi(x), x) + (\theta*n(x)*Derivative(\eta^{n+1}(x), x)*Derivative(u^{n+1}(x), x) - \theta*n(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) - \theta*n(x)*Derivative(b(x), x)*Derivative(u^{n+1}(x), x) + \theta*n(x)*Derivative(b(x), x)*Derivative(u^{n}(x), x) + \theta*Derivative(\eta^{n+1}(x), (x, 2))*Derivative(u^{n+1}(x), x) - \theta*Derivative(\eta^{n}(x), (x, 2))*Derivative(u^{n}(x), x) - \theta*Derivative(b(x), (x, 2))*Derivative(u^{n+1}(x), x) + \theta*Derivative(b(x), (x, 2))*Derivative(u^{n}(x), x) + n(x)*Derivative(\eta^{n}(x), x)*Derivative(u^{n}(x), x) - n(x)*Derivative(b(